In [3]:
cd drive/'My Drive'/'Colab Notebooks'/Road_seg

/content/drive/My Drive/Colab Notebooks/Road_seg


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf

from tensorflow import keras
import os,sys
import albumentations as albu
from PIL import Image

%load_ext autoreload
%autoreload 2

# Loading the dataset

In [2]:
import helper

Using TensorFlow backend.


In [4]:
def patcher_3000(img, cp_size, p_size):
    list_patches = []
    padding = int((p_size - cp_size) / 2)
    padded_img = np.pad(img, ((padding, padding),(padding, padding), (0,0)), mode='symmetric')

    imgwidth = img.shape[0]
    imgheight = img.shape[1]

    for i in range(0, imgheight, cp_size):
        for j in range(0, imgwidth, cp_size):
            im_patch = padded_img[j:j+cp_size+2*padding, i:i+cp_size+2*padding, :]
            list_patches.append(im_patch)
            
    
    return list_patches

In [5]:
# Loaded a set of images
root_dir = "Datasets/training/"

n = 100

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
print("Loading " + str(n) + " images")
imgs = [helper.load_image(image_dir + files[i]) for i in range(n)]

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [helper.load_image(gt_dir + files[i]) for i in range(n)]

Loading 100 images
Loading 100 images


#### Data Augmentation
##### From https://towardsdatascience.com/road-detection-using-segmentation-models-and-albumentations-libraries-on-keras-d5434eaf73a8


In [ ]:
#Doubles the nb of images. Needs improvement
#for i in range(2):
  for n in range(len(gt_imgs)):
    augmented = helperC.image_augmentation(image_size = 256)(image=imgs[n], mask=gt_imgs[n])
    imgs.append(augmented['image'])
    gt_imgs.append(augmented['mask'])

In [ ]:
# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels
window_size=100
img_patches = [patcher_3000(imgs[i], patch_size, window_size) for i in range(len(imgs))]
gt_patches = [helperC.img_crop(gt_imgs[i], patch_size, patch_size) for i in range(len(gt_imgs))]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])
gt_patches = helperC.patches_labelization(gt_patches)

In [10]:
print(img_patches.shape)

(125000, 100, 100, 3)


## Feature processing 

#### Balancing amount of patches  background vs road for training

In [ ]:

#img_patches, gt_patches = helper.feature_balancing(img_patches, gt_patches)
#Works badly!

# CNN

In [ ]:
from tensorflow.keras import datasets, layers, models

## Layers

In [13]:
from keras.regularizers import l2
"""model = models.Sequential()
reg = 1e-6 # L2 regularization factor (used on weights, but not biases)
### heavily(!) inspired from a previous project. Needs to be changed/optimized
model.add(layers.Convolution2D(64, 5, 5, padding='same',input_shape=(16, 16, 3)))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(layers.Dropout(0.25))
model.add(layers.Convolution2D(128, 3, 3,padding='same'))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(layers.Dropout(0.25))
model.add(layers.Convolution2D(256, 3, 3, padding='same'))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(layers.Dropout(0.25))
model.add(layers.Convolution2D(256, 3, 3, padding='same'))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D(pool_size=(2,2), padding='same'))
model.add(layers.Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(128, kernel_regularizer=l2(reg))) # Fully connected layer (128 neurons)
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, kernel_regularizer=l2(reg)))
"""

"""model = models.Sequential()

model.add(layers.Conv2D(32, (2, 2), input_shape=(16, 16, 3)))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (2, 2)))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Conv2D(128, (2, 2)))
model.add(layers.LeakyReLU(alpha=0.1))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64))
model.add(layers.LeakyReLU(alpha=0.1))
model.add(layers.Dense(2, activation='softmax'))"""
model = models.Sequential()
lrelu = lambda x: tf.keras.layers.LeakyReLU(alpha=0.1)(x)
kernel_size3 = (3,3)
kernel_size2 = (2,2)
pool_size = (2,2)
alpha_relu = 0.1
regularizer = 1e-6
shape = (window_size, window_size, 3)

model.add(layers.Convolution2D(64,kernel_size2,activation = lrelu, input_shape=shape))
model.add(layers.MaxPooling2D(pool_size))
model.add(layers.Convolution2D(64, kernel_size2,activation = lrelu))
model.add(layers.Dropout(0.1))
model.add(layers.MaxPooling2D(pool_size))

model.add(layers.Convolution2D(128, kernel_size2,activation = lrelu))
model.add(layers.MaxPooling2D(pool_size))
model.add(layers.Convolution2D(128,kernel_size2,activation = lrelu))
model.add(layers.Dropout(0.1))
model.add(layers.MaxPooling2D(pool_size))

model.add(layers.Convolution2D(256,kernel_size2,activation = lrelu))
model.add(layers.MaxPooling2D(pool_size))
model.add(layers.Convolution2D(256,kernel_size2,activation = lrelu))
model.add(layers.Dropout(0.1))

model.add(layers.Flatten())
model.add(layers.Dense(2, activation='softmax'))


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Using TensorFlow backend.


In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 99, 99, 64)        832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 49, 49, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 64)        16448     
_________________________________________________________________
dropout (Dropout)            (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 128)       32896     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 128)       0

## Training

In [ ]:

def generate_minibatch(X, Y, batch_size):
    """
    Procedure for real-time minibatch creation and image augmentation.
     This runs in a parallel thread while the model is being trained.
    """
    Y = tf.keras.utils.to_categorical(Y, 2)
    while 1:
        # Generate one minibatch
        X_batch = np.empty((batch_size, window_size,window_size, 3))
        Y_batch = np.empty((batch_size, 2))
        for i in range(batch_size):
            # Select a random image
            idx = np.random.choice(X.shape[0])
                      
            # The label does not depend on the image rotation/flip (provided that the rotation is in steps of 90°)
            X_batch[i] = X[idx]
            Y_batch[i] = Y[idx]
        yield (X_batch, Y_batch)
          

In [ ]:
import math

batch_size = 64
steps_per_epoch = math.ceil(img_patches.shape[0] / batch_size)
"""def softmax_categorical_crossentropy(y_true, y_pred):
       
        Uses categorical cross-entropy from logits in order to improve numerical stability.
        This is especially useful for TensorFlow (less useful for Theano).           
        return K.categorical_crossentropy(y_pred, y_true, from_logits=True)
    """
model.compile(loss = "categorical_crossentropy",  optimizer = keras.optimizers.Adam(lr=1e-3),
              metrics = ['accuracy'])
#Y = tf.keras.utils.to_categorical(gt_patches, 2)

        
#model.fit(img_patches, Y, batch_size = 64, epochs = 5, validation_split = 0.2)
model.fit_generator(generate_minibatch(img_patches,gt_patches, batch_size),
                    epochs=25, steps_per_epoch= steps_per_epoch)


Epoch 1/25
1954/1954 [==============================] - 56s 29ms/step - loss: 0.3138 - acc: 0.8560
Epoch 2/25
1954/1954 [==============================] - 52s 27ms/step - loss: 0.1426 - acc: 0.9435
Epoch 3/25
1954/1954 [==============================] - 52s 27ms/step - loss: 0.0972 - acc: 0.9628
Epoch 4/25
1954/1954 [==============================] - 52s 27ms/step - loss: 0.0727 - acc: 0.9727
Epoch 5/25
1954/1954 [==============================] - 52s 27ms/step - loss: 0.0589 - acc: 0.9785
Epoch 6/25
1954/1954 [==============================] - 53s 27ms/step - loss: 0.0505 - acc: 0.9820
Epoch 7/25
1954/1954 [==============================] - 52s 27ms/step - loss: 0.0438 - acc: 0.9843
Epoch 8/25
1954/1954 [==============================] - 52s 27ms/step - loss: 0.0410 - acc: 0.9855
Epoch 9/25
1954/1954 [==============================] - 52s 27ms/step - loss: 0.0370 - acc: 0.9869
Epoch 10/25
1954/1954 [==============================] - 52s 27ms/step - loss: 0.0337 - acc: 0.9884
Epoch 11/

In [ ]:
weights = model.get_weights()  # Retrieves the state of the model.
#model.set_weights(weights)

## Predictions

In [ ]:
#predictions = model.predict(img_patches)
#predictions = (predictions[:,0] < predictions[:,1]) * 1
#predictions = np.squeeze(predictions)

In [ ]:
#predictions = np.squeeze(helper.binarize_predictions(predictions))

In [ ]:
#np.set_printoptions(threshold=sys.maxsize)
#print(predictions)

# Displaying predictions

In [ ]:
"""# Display prediction as an image
img_idx = 2

img_indexed = imgs[img_idx]
img_pat = helper.img_crop(img_indexed, patch_size, patch_size)
img_pat = np.asarray(img_pat)
prediction = model.predict(img_pat)

w = gt_imgs[img_idx].shape[0]
h = gt_imgs[img_idx].shape[1]
predicted_im = helper.label_to_img_array(w, h, patch_size, patch_size, helper.binarize_predictions(prediction))
cimg = helper.concatenate_images(imgs[img_idx], predicted_im)
fig1 = plt.figure(figsize=(10, 10)) # create a figure with the default size 
plt.imshow(cimg, cmap='Greys_r')

new_img = helper.make_img_overlay(imgs[img_idx], predicted_im)

#plt.imshow(new_img)"""

# Create Submission

In [6]:
import submission as sub

In [9]:
n = 50
patch_size = 16 
window_size = 100
img_test = []

for i in range(1, 51):
    image_filename = 'Datasets/test_set_images/test_' + str(i) + '/test_' + str(i) + '.png' 
    img_test.append(helper.load_image(image_filename))

    img_patches_test = [patcher_3000(img_test[i], patch_size, window_size) for i in range(n)]
img_patches_test = np.asarray([img_patches_test[i][j] for i in range(len(img_patches_test))
                               for j in range(len(img_patches_test[i]))])

IndexError: list index out of range

In [ ]:
new_model = keras.models.load_model('saved_M1_fit_SoftmaxCategorical_shuffle_15.h5')

In [ ]:
predictions = new_model.predict(img_patches_test)
#predictions = np.squeeze(helper.binarize_predictions(predictions))
print(predictions.shape)
#print(predictions[10])


In [ ]:


"""np.set_printoptions(threshold=sys.maxsize)
print(predictions.shape)
print(predictions)
"""

### Post-processing

In [ ]:
import cv2
def morphological(img):
    kernel = np.ones((17,17),np.uint8)
    op = cv2.morphologyEx(img,cv2.MORPH_OPEN, kernel)
    cl = cv2.morphologyEx(op,cv2.MORPH_CLOSE,kernel)   
    return cl

In [ ]:

prediction_patches =[]
#predictions = np.squeeze(helper.binarize_predictions(predictions))
predictions = (predictions[:,0] < predictions[:,1]) * 1
predictions = np.squeeze(predictions)
for i in range(50):
    #predictions = np.squeeze(helper.binarize_predictions(predictions))
    #prediction = helper.binarize_predictions_array(predictions[i*1444 :((i*1444)+1444)])
    prediction = helperC.label_to_img_patches_test(predictions[i*1444 :((i*1444)+1444)])
    #prediction = helperC.combine_surounded_patches(prediction) #Which order? Is it even useful. 
    #prediction = helperC.remove_lonely_patches(prediction)
    prediction = morphological(prediction)
    #prediction = morphological(prediction)
    prediction_patches.append(prediction) 


In [ ]:

img_patches_test_mod = [helperC.img_crop(prediction_patches[i], 1, 1)
                        for i in range(len(prediction_patches))]

img_patches_test_mod =  np.asarray([img_patches_test_mod[i][j] for i in range(len(img_patches_test_mod))
                                     for j in range(len(img_patches_test_mod[i]))])

"""img_patches_test_mod = np.asarray([prediction_patches[i][j] for i in range(len(prediction_patches)) 
                               for j in range(len(prediction_patches[i]))])"""

predictions_mod = np.squeeze(img_patches_test_mod)

sub.create_pred_images(predictions_mod)


In [ ]:
submission_filename = 'submission3.csv'
   
sub.create_submission(submission_filename)

In [ ]:
#Just to have a look..
print(img_patches_test_mod.shape)
print(img_patches_test.shape)
for i in range(25):
    fig1 = plt.figure(figsize=(5, 5))
    plt.imshow(prediction_patches[i], cmap='Greys_r')

In [ ]:
for i in range(25):
    fig1 = plt.figure(figsize=(5, 5))
    plt.imshow(img_test[i])